# ***Upload and Load dataset***

In [ ]:
#Upload the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/UM-Internship/HRDataset.csv')

# ***Data Cleaning***

In [ ]:
#Display basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

In [ ]:
#Identify missing values
df.isnull().sum()

,0
Employee_Name,0
EmpID,0
MarriedID,0
MaritalStatusID,0
GenderID,0
EmpStatusID,0
DeptID,0
PerfScoreID,0
FromDiversityJobFairID,0
Salary,0


In [ ]:
#Identify Duplicate rows
df.duplicated().sum()

np.int64(0)

In [ ]:
#Drop columns
cols_to_drop = [
    'EmpID', 'MarriedID', 'MaritalStatusID', 'GenderID',
    'EmpStatusID', 'DeptID', 'PerfScoreID', 'Termd',
    'PositionID', 'ManagerID', 'Zip', 'FromDiversityJobFairID',
    'CitizenDesc', 'HispanicLatino', 'RaceDesc'
]

df = df.drop(columns=cols_to_drop)

In [ ]:
#Change datatype
df['DOB'] = pd.to_datetime(df['DOB'], format='%m/%d/%y')
df['DOB'] = df['DOB'].mask(df['DOB'] > '2005-01-01', df['DOB'] - pd.DateOffset(years=100))

df['DateofHire'] = pd.to_datetime(df['DateofHire'], format='%m/%d/%Y')

df['DateofTermination'] = pd.to_datetime(
    df['DateofTermination'],
    format='%m/%d/%Y',
    errors='coerce'
)

df['LastPerformanceReview_Date'] = pd.to_datetime(
    df['LastPerformanceReview_Date'],
    format='%m/%d/%Y',
    errors='coerce'
)

In [ ]:
# Create variable age
today = pd.to_datetime('2025-01-01')
df['Age'] = (today - df['DOB']).dt.days // 365

In [ ]:
#Create variable TerminatedFlag
df['TerminatedFlag'] = df['DateofTermination'].notna().astype(int)

In [ ]:
# Create employee tenure
df['Tenure_Years'] = (
    df['DateofTermination'].fillna(today) - df['DateofHire']
).dt.days / 365

In [ ]:
#Create Days_Since_Last_Review variable
df['Days_Since_Last_Review'] = (today - df['LastPerformanceReview_Date']).dt.days

In [ ]:
#Create Team_Size variable
df['Team_Size'] = df.groupby('ManagerName')['Employee_Name'].transform('count')

In [ ]:
#Season of Hire
def season(x):
    m = x.month
    if m in [12,1,2]: return 'Winter'
    if m in [3,4,5]: return 'Spring'
    if m in [6,7,8]: return 'Summer'
    return 'Autumn'

df['Hire_Season'] = df['DateofHire'].apply(season)
print(df['Hire_Season'].head())

0    Summer
1    Spring
2    Summer
3    Winter
4    Summer
Name: Hire_Season, dtype: object


# ***Data Understanding***

In [ ]:
#Download the cleaned dataset file
df.to_csv('HRDataset_cleaned.csv', index=False)

In [ ]:
from google.colab import files
files.download('HRDataset_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Display final dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Employee_Name               311 non-null    object        
 1   Salary                      311 non-null    int64         
 2   Position                    311 non-null    object        
 3   State                       311 non-null    object        
 4   DOB                         311 non-null    datetime64[ns]
 5   Sex                         311 non-null    object        
 6   MaritalDesc                 311 non-null    object        
 7   DateofHire                  311 non-null    datetime64[ns]
 8   DateofTermination           104 non-null    datetime64[ns]
 9   TermReason                  311 non-null    object        
 10  EmploymentStatus            311 non-null    object        
 11  Department                  311 non-null    object        

In [ ]:
#Display summary statistics (numerical)
numeric_cols = [
    'Salary',
    'EngagementSurvey',
    'EmpSatisfaction',
    'SpecialProjectsCount',
    'DaysLateLast30',
    'Absences',
    'Age',
    'TerminatedFlag',
    'Tenure_Years',
    'Days_Since_Last_Review',
    'Team_Size'
]

df[numeric_cols].describe()

,Salary,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,Age,TerminatedFlag,Tenure_Years,Days_Since_Last_Review,Team_Size
count,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000
mean,69020.684887,4.110000,3.890675,1.218650,0.414791,10.237942,45.446945,0.334405,8.719103,2669.112540,18.221865
std,25156.636930,0.789938,0.909241,2.349421,1.294519,5.852596,8.899959,0.472542,4.219209,827.612978,5.237486
min,45046.000000,1.120000,1.000000,0.000000,0.000000,1.000000,32.000000,0.000000,0.071233,2134.000000,2.000000
25%,55501.500000,3.690000,3.000000,0.000000,0.000000,5.000000,38.000000,0.000000,4.965753,2158.000000,14.000000
50%,62810.000000,4.280000,4.000000,0.000000,0.000000,10.000000,44.000000,0.000000,10.150685,2177.000000,21.000000
75%,72036.000000,4.700000,5.000000,0.000000,0.000000,15.000000,51.000000,1.000000,11.493151,3209.500000,22.000000
max,250000.000000,5.000000,5.000000,8.000000,6.000000,20.000000,74.000000,1.000000,18.991781,5285.000000,22.000000


In [ ]:
#Display summary statistics (Categorical)
categorical_cols = [
    'Position', 'State', 'Sex', 'MaritalDesc',
    'Department', 'ManagerName', 'RecruitmentSource', 'PerformanceScore',
    'Hire_Season'
]

df[categorical_cols].describe().transpose()

,count,unique,top,freq
Position,311,32,Production Technician I,137
State,311,28,MA,276
Sex,311,2,F,176
MaritalDesc,311,5,Single,137
Department,311,6,Production,209
ManagerName,311,21,Michael Albert,22
RecruitmentSource,311,9,Indeed,87
PerformanceScore,311,4,Fully Meets,243
Hire_Season,311,4,Winter,86


In [ ]:
#Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# ***Statistical Validation***

In [ ]:
#Chi-test
import pandas as pd
from scipy.stats import chi2_contingency
from itertools import combinations

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

def chi_square_test(df, col1, col2):
    table = pd.crosstab(df[col1], df[col2])

    if table.shape[0] < 2 or table.shape[1] < 2:
        return None

    chi2, p, dof, expected = chi2_contingency(table)
    return p

results = []

for col1, col2 in combinations(cat_cols, 2):
    p_value = chi_square_test(df, col1, col2)

    if p_value is not None:
        results.append([col1, col2, p_value])

chi_df = pd.DataFrame(results, columns=['Variable 1', 'Variable 2', 'p-value'])

significant_chi = chi_df[chi_df['p-value'] < 0.05].sort_values('p-value')
significant_chi

,Variable 1,Variable 2,p-value
17,Position,ManagerName,0.000000e+00
16,Position,Department,2.002489e-229
56,Department,ManagerName,1.586280e-202
45,TermReason,EmploymentStatus,1.299847e-101
25,State,Department,4.736785e-14
26,State,ManagerName,9.468476e-12
60,ManagerName,RecruitmentSource,1.618008e-11
57,Department,RecruitmentSource,1.348631e-08
53,EmploymentStatus,RecruitmentSource,9.920013e-06
18,Position,RecruitmentSource,7.013064e-05


In [ ]:
#ANOVA test
import pandas as pd
import scipy.stats as stats

numeric_cols = [
    'Salary', 'EngagementSurvey', 'EmpSatisfaction',
    'SpecialProjectsCount', 'DaysLateLast30', 'Absences',
    'Age', 'Tenure_Years', 'Days_Since_Last_Review', 'Team_Size'
]

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

def run_clean_anova(df, cat_col, num_col):
    valid_groups = []

    for level in df[cat_col].dropna().unique():
        group = df[df[cat_col] == level][num_col].dropna()

        if len(group) >= 2 and group.var() > 0:
            valid_groups.append(group)

    if len(valid_groups) < 2:
        return None

    try:
        f_stat, p_val = stats.f_oneway(*valid_groups)
        return p_val
    except:
        return None

anova_results = []

for cat in categorical_cols:
    for num in numeric_cols:
        p = run_clean_anova(df, cat, num)
        if p is not None:
            anova_results.append([cat, num, p])

anova_df = pd.DataFrame(anova_results, columns=['Categorical_Var', 'Numeric_Var', 'p-value'])
significant_anova = anova_df[anova_df['p-value'] < 0.05].sort_values('p-value')
significant_anova

,Categorical_Var,Numeric_Var,p-value
63,Department,SpecialProjectsCount,3.835128e-163
9,Position,Team_Size,8.925870e-126
93,PerformanceScore,DaysLateLast30,5.744323e-114
0,Position,Salary,2.034359e-107
3,Position,SpecialProjectsCount,8.338050e-106
57,EmploymentStatus,Tenure_Years,1.216967e-102
69,Department,Team_Size,1.873112e-99
58,EmploymentStatus,Days_Since_Last_Review,8.556718e-97
47,TermReason,Tenure_Years,2.591518e-89
48,TermReason,Days_Since_Last_Review,2.104359e-86


In [ ]:
#Tukey Post - Hoc
significant_pairs = significant_anova[['Categorical_Var', 'Numeric_Var']].values.tolist()

from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

def get_valid_groups(df, cat, num):
    groups = []
    labels = []

    for level in df[cat].dropna().unique():
        group_data = df.loc[df[cat] == level, num].dropna()

        if len(group_data) >= 2 and group_data.var() > 0:
            groups.append(group_data)
            labels.append(np.repeat(level, len(group_data)))

    if len(groups) < 2:
        return None, None

    data = pd.concat(groups)
    labels = np.concatenate(labels)

    return data, labels

tukey_results = {}

for cat, num in significant_pairs:
    data, labels = get_valid_groups(df, cat, num)

    if data is None:
        continue

    try:
        tukey = pairwise_tukeyhsd(endog=data, groups=labels, alpha=0.05)
        tukey_results[(cat, num)] = tukey
    except Exception as e:
        pass

significant_tukey_only = {}

for (cat, num), tukey in tukey_results.items():
    df_tukey = pd.DataFrame(
        data=tukey._results_table.data[1:],
        columns=tukey._results_table.data[0]
    )

    df_sig = df_tukey[df_tukey['reject'] == True]

    if not df_sig.empty:
        significant_tukey_only[(cat, num)] = df_sig

for (cat, num), df_sig in significant_tukey_only.items():
    print("\n" + "="*100)
    print(f"Significant Tukey Results — {cat} → {num}")
    print("="*100)
    print(df_sig)


Significant Tukey Results — Department → SpecialProjectsCount
              group1                group2  meandiff  p-adj   lower   upper  \
0      Admin Offices                 IT/IS    2.2533    0.0  1.6689  2.8378   
1      Admin Offices     Production          -3.6475    0.0 -4.1970 -3.0980   
3              IT/IS     Production          -5.9009    0.0 -6.1550 -5.6467   
4              IT/IS  Software Engineering   -1.7382    0.0 -2.2757 -1.2006   
5  Production         Software Engineering    4.1627    0.0  3.6634  4.6620   

   reject  
0    True  
1    True  
3    True  
4    True  
5    True  

Significant Tukey Results — Position → Team_Size
                      group1                    group2  meandiff   p-adj  \
0         Area Sales Manager                IT Support   -4.5185  0.0000   
1         Area Sales Manager   Production Technician I    8.0435  0.0000   
2         Area Sales Manager  Production Technician II    8.0604  0.0000   
3         Area Sales Manager        

In [ ]:
#ANOVA Two way
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import itertools
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

cat_cols_filtered = []
for c in cat_cols:
    if 2 <= df[c].nunique() <= 10:
        if df[c].value_counts().min() >= 5:
            cat_cols_filtered.append(c)

print("Using categorical variables:", cat_cols_filtered)

significant_results = []

print("\nRunning Two-Way ANOVA automatically...\n")

for num in num_cols:
    for cat1, cat2 in itertools.combinations(cat_cols_filtered, 2):

        formula = f"{num} ~ C({cat1}) + C({cat2}) + C({cat1}):C({cat2})"

        try:
            model = ols(formula, data=df).fit()
            anova = sm.stats.anova_lm(model, typ=2)

            p1 = anova.loc[f"C({cat1})", "PR(>F)"]
            p2 = anova.loc[f"C({cat2})", "PR(>F)"]
            p_int = anova.loc[f"C({cat1}):C({cat2})", "PR(>F)"]

            if p1 < 0.05 or p2 < 0.05 or p_int < 0.05:
                significant_results.append({
                    "Numeric Variable": num,
                    "Factor 1": cat1,
                    "Factor 2": cat2,
                    "p(Factor1)": p1,
                    "p(Factor2)": p2,
                    "p(Interaction)": p_int
                })

        except Exception as e:
            continue

if significant_results:
    print("SIGNIFICANT TWO-WAY ANOVA RESULTS:\n")
    for r in significant_results:
        print(f"{r['Numeric Variable']} ~ {r['Factor 1']} + {r['Factor 2']}")
        print(f" → p({r['Factor 1']}): {r['p(Factor1)']:.5f}")
        print(f" → p({r['Factor 2']}): {r['p(Factor2)']:.5f}")
        print(f" → p(Interaction): {r['p(Interaction)']:.5f}")
        print("-" * 60)
else:
    print("No significant Two-Way ANOVA results found.")

Using categorical variables: ['Sex', 'MaritalDesc', 'EmploymentStatus', 'PerformanceScore', 'Hire_Season']

Running Two-Way ANOVA automatically...

SIGNIFICANT TWO-WAY ANOVA RESULTS:

Salary ~ Sex + Hire_Season
 → p(Sex): 0.35661
 → p(Hire_Season): 0.04473
 → p(Interaction): 0.06161
------------------------------------------------------------
Salary ~ MaritalDesc + PerformanceScore
 → p(MaritalDesc): 0.33004
 → p(PerformanceScore): 0.64157
 → p(Interaction): 0.00540
------------------------------------------------------------
Salary ~ MaritalDesc + Hire_Season
 → p(MaritalDesc): 0.54714
 → p(Hire_Season): 0.01879
 → p(Interaction): 0.95304
------------------------------------------------------------
Salary ~ EmploymentStatus + Hire_Season
 → p(EmploymentStatus): 0.06485
 → p(Hire_Season): 0.04506
 → p(Interaction): 0.94113
------------------------------------------------------------
Salary ~ PerformanceScore + Hire_Season
 → p(PerformanceScore): 0.10736
 → p(Hire_Season): 0.04701
 → p(

# ***Business Insights***

In [ ]:
#Salary × EmploymentStatus × Department × TerminatedFlag
df.groupby(['EmploymentStatus', 'Department', 'TerminatedFlag'])['Salary'].agg(
    Count='count',
    Avg_Salary1='mean',
    Min_Salary1='min',
    Max_Salary1='max',
    Median_Salary1='median'
).reset_index()

,EmploymentStatus,Department,TerminatedFlag,Count,Avg_Salary1,Min_Salary1,Max_Salary1,Median_Salary1
0,Active,Admin Offices,0,7,77315.285714,51920,106367,64520.0
1,Active,Executive Office,0,1,250000.000000,250000,250000,250000.0
2,Active,IT/IS,0,40,94512.075000,50178,220450,89587.5
3,Active,Production,0,126,60419.261905,45046,170500,60441.0
4,Active,Sales,0,26,69088.846154,55875,180000,64046.0
5,Active,Software Engineering,0,7,93262.571429,77692,105688,93396.0
6,Terminated for Cause,Admin Offices,1,1,49920.000000,49920,49920,49920.0
7,Terminated for Cause,IT/IS,1,4,110603.000000,103613,114800,111999.5
8,Terminated for Cause,Production,1,8,60383.625000,49773,82758,59134.0
9,Terminated for Cause,Sales,1,2,64778.500000,59370,70187,64778.5


In [ ]:
#Salary × PerformanceScore × Department × TerminatedFlag
df.groupby(["PerformanceScore", "Department", "TerminatedFlag"]).agg(
          Employee_Count = ("Salary", "count"),
          Avg_Salary2 = ("Salary", "mean"),
          Min_Salary2 = ("Salary", "min"),
          Median_Salary2 = ("Salary", "median"),
          Max_Salary2 = ("Salary", "max")
      ).reset_index().sort_values(by=["Department", "PerformanceScore", "TerminatedFlag"])

,PerformanceScore,Department,TerminatedFlag,Employee_Count,Avg_Salary2,Min_Salary2,Median_Salary2,Max_Salary2
6,Fully Meets,Admin Offices,0,7,77315.285714,51920,64520.0,106367
7,Fully Meets,Admin Offices,1,2,52460.000000,49920,52460.0,55000
8,Fully Meets,Executive Office,0,1,250000.000000,250000,250000.0,250000
0,Exceeds,IT/IS,0,6,129166.166667,51777,116221.0,220450
9,Fully Meets,IT/IS,0,32,87347.500000,50178,87873.5,150290
10,Fully Meets,IT/IS,1,10,107274.900000,75281,107218.5,148999
17,Needs Improvement,IT/IS,0,1,157000.000000,157000,157000.0,157000
22,PIP,IT/IS,0,1,53366.000000,53366,53366.0,53366
1,Exceeds,Production,0,20,67264.800000,46738,62285.5,170500
2,Exceeds,Production,1,7,56207.571429,46837,57954.0,70507


In [ ]:
#Salary × Sex × PerformanceScore × TerminatedFlag
df.groupby(['Sex', 'PerformanceScore', 'TerminatedFlag']) \
                  .agg(
                      Count=('Salary', 'count'),
                      Avg_Salary3=('Salary', 'mean'),
                     Min_Salary3=('Salary', 'min'),
                      Max_Salary3=('Salary', 'max'),
                      Median_Salary3=('Salary', 'median')
                  ) \
                  .reset_index()

,Sex,PerformanceScore,TerminatedFlag,Count,Avg_Salary3,Min_Salary3,Max_Salary3,Median_Salary3
0,F,Exceeds,0,16,74874.000000,46738,220450,63116.0
1,F,Exceeds,1,4,54396.500000,47434,62425,53863.5
2,F,Fully Meets,0,93,70194.870968,45046,250000,62514.0
3,F,Fully Meets,1,49,64006.428571,45433,120000,62659.0
4,F,Needs Improvement,0,4,59700.000000,47750,66441,62304.5
5,F,Needs Improvement,1,5,66074.000000,50274,99280,60270.0
6,F,PIP,0,3,60033.000000,56847,64021,59231.0
7,F,PIP,1,2,50593.500000,48513,52674,50593.5
8,M,Exceeds,0,13,88764.307692,47414,178000,64397.0
9,M,Exceeds,1,4,71213.500000,46837,108987,64515.0


In [ ]:
#Salary × MaritalDesc × PerformanceScore × TerminatedFlag
df.groupby(
    ["MaritalDesc", "PerformanceScore", "TerminatedFlag"]).agg(
    Count=("Salary", "count"),
    Average_Salary4=("Salary", "mean"),
    Median_Salary4=("Salary", "median"),
    Min_Salary4=("Salary", "min"),
    Max_Salary4=("Salary", "max")
).reset_index()

,MaritalDesc,PerformanceScore,TerminatedFlag,Count,Average_Salary4,Median_Salary4,Min_Salary4,Max_Salary4
0,Divorced,Exceeds,0,3,70981.333333,60724.0,59892,92328
1,Divorced,Exceeds,1,2,60474.000000,60474.0,58523,62425
2,Divorced,Fully Meets,0,9,60483.222222,62957.0,45069,88976
3,Divorced,Fully Meets,1,14,65252.571429,59895.5,45115,148999
4,Divorced,Needs Improvement,0,2,70521.500000,70521.5,68051,72992
5,Married,Exceeds,0,8,63887.500000,62487.5,46738,93554
6,Married,Exceeds,1,3,51521.333333,49773.0,46837,57954
7,Married,Fully Meets,0,63,73462.174603,63381.0,46120,250000
8,Married,Fully Meets,1,37,65098.918919,61154.0,45433,113999
9,Married,Needs Improvement,0,2,109292.000000,109292.0,61584,157000


In [ ]:
#Salary × Sex × Department × TerminatedFlag
df.groupby(["Sex", "Department", "TerminatedFlag"]).agg(
    Count=("Salary", "count"),
    Avg_Salary5=("Salary", "mean"),
    Median_Salary5=("Salary", "median"),
    Min_Salary5=("Salary", "min"),
    Max_Salary5=("Salary", "max")
).reset_index()

,Sex,Department,TerminatedFlag,Count,Avg_Salary5,Median_Salary5,Min_Salary5,Max_Salary5
0,F,Admin Offices,0,4,80159.500000,81175.5,51920,106367
1,F,Admin Offices,1,2,52460.000000,52460.0,49920,55000
2,F,Executive Office,0,1,250000.000000,250000.0,250000,250000
3,F,IT/IS,0,19,91381.368421,89292.0,50750,220450
4,F,IT/IS,1,3,114933.333333,114800.0,110000,120000
5,F,Production,0,75,59712.133333,60724.0,45046,77915
6,F,Production,1,51,59485.549020,60070.0,45433,80512
7,F,Sales,0,12,73129.583333,62447.5,57859,180000
8,F,Sales,1,3,67565.000000,68999.0,59370,74326
9,F,Software Engineering,0,5,96431.400000,95660.0,86214,105688


In [ ]:
#Salary × EmploymentStatus × PerformanceScore × TerminatedFlag
df.groupby(
    ["EmploymentStatus", "PerformanceScore", "TerminatedFlag"]
).agg(
    Count=("Salary", "count"),
    Avg_Salary6=("Salary", "mean"),
    Median_Salary6=("Salary", "median"),
    Min_Salary6=("Salary", "min"),
    Max_Salary6=("Salary", "max")
).reset_index()

,EmploymentStatus,PerformanceScore,TerminatedFlag,Count,Avg_Salary6,Median_Salary6,Min_Salary6,Max_Salary6
0,Active,Exceeds,0,29,81100.689655,63353.0,46738,220450
1,Active,Fully Meets,0,162,69267.222222,63079.5,45046,250000
2,Active,Needs Improvement,0,8,73737.875000,64733.0,47750,157000
3,Active,PIP,0,8,58819.000000,58752.0,53189,64057
4,Terminated for Cause,Exceeds,1,1,49773.000000,49773.0,49773,49773
5,Terminated for Cause,Fully Meets,1,8,86536.750000,93185.5,49920,114800
6,Terminated for Cause,Needs Improvement,1,5,67862.000000,60340.0,59124,99280
7,Terminated for Cause,PIP,1,2,61430.500000,61430.5,52674,70187
8,Voluntarily Terminated,Exceeds,1,7,64666.714286,58523.0,46837,108987
9,Voluntarily Terminated,Fully Meets,1,73,64377.109589,61729.0,45115,148999


In [ ]:
#Salary × Department × PerformanceScore × TerminatedFlag
df.groupby(
    ["Department", "PerformanceScore", "TerminatedFlag"]
).agg(
    Count=("Salary", "count"),
    Avg_Salary7=("Salary", "mean"),
    Median_Salary7=("Salary", "min"),
    Min_Salary7=("Salary", "min"),
    Max_Salary7=("Salary", "max")
).reset_index()

,Department,PerformanceScore,TerminatedFlag,Count,Avg_Salary7,Median_Salary7,Min_Salary7,Max_Salary7
0,Admin Offices,Fully Meets,0,7,77315.285714,51920,51920,106367
1,Admin Offices,Fully Meets,1,2,52460.000000,49920,49920,55000
2,Executive Office,Fully Meets,0,1,250000.000000,250000,250000,250000
3,IT/IS,Exceeds,0,6,129166.166667,51777,51777,220450
4,IT/IS,Fully Meets,0,32,87347.500000,50178,50178,150290
5,IT/IS,Fully Meets,1,10,107274.900000,75281,75281,148999
6,IT/IS,Needs Improvement,0,1,157000.000000,157000,157000,157000
7,IT/IS,PIP,0,1,53366.000000,53366,53366,53366
8,Production,Exceeds,0,20,67264.800000,46738,46738,170500
9,Production,Exceeds,1,7,56207.571429,46837,46837,70507


In [ ]:
#Salary x TermReason
df.groupby(['TermReason']).agg(
    Avg_Salary22=('Salary', 'mean')
).reset_index()

,TermReason,Avg_Salary22
0,Another position,66116.900000
1,Fatal attraction,70187.000000
2,Learned that he is a gangster,103613.000000
3,N/A-StillEmployed,70694.033816
4,attendance,67347.428571
5,career change,65409.444444
6,gross misconduct,49773.000000
7,hours,75731.625000
8,maternity leave - did not return,63800.000000
9,medical issues,73898.333333


In [ ]:
#Position × PerformanceScore × Salary × TerminatedFlag
df.groupby(
    ["Position", "PerformanceScore", "TerminatedFlag"]
).agg(
    Count=("Salary", "count"),
    Avg_Salary8=("Salary", "mean"),
    Median_Salary8=("Salary", "median"),
    Min_Salary8=("Salary", "min"),
    Max_Salary8=("Salary", "max")
).reset_index()

,Position,PerformanceScore,TerminatedFlag,Count,Avg_Salary8,Median_Salary8,Min_Salary8,Max_Salary8
0,Accountant I,Fully Meets,0,3,63507.666667,63003.0,63000,64520
1,Administrative Assistant,Fully Meets,0,1,51920.000000,51920.0,51920,51920
2,Administrative Assistant,Fully Meets,1,2,52460.000000,52460.0,49920,55000
3,Area Sales Manager,Exceeds,0,2,69319.000000,69319.0,64397,74241
4,Area Sales Manager,Fully Meets,0,18,64437.722222,63493.0,55875,71860
5,Area Sales Manager,Fully Meets,1,3,68467.666667,71707.0,59370,74326
6,Area Sales Manager,PIP,0,3,59690.666667,59231.0,58273,61568
7,Area Sales Manager,PIP,1,1,70187.000000,70187.0,70187,70187
8,BI Developer,Fully Meets,0,4,95465.000000,96370.0,90100,99020
9,BI Director,Fully Meets,0,1,110929.000000,110929.0,110929,110929


In [ ]:
#Position × PerformanceScore × Department × TerminatedFlag
df.pivot_table(
    index = ['Position', 'PerformanceScore', 'Department'],
    columns = 'TerminatedFlag',
    aggfunc = 'size',
    fill_value = 0
).reset_index()

TerminatedFlag,Position,PerformanceScore,Department,0,1
0,Accountant I,Fully Meets,Admin Offices,3,0
1,Administrative Assistant,Fully Meets,Admin Offices,1,2
2,Area Sales Manager,Exceeds,Sales,2,0
3,Area Sales Manager,Fully Meets,Sales,18,3
4,Area Sales Manager,PIP,Sales,3,1
5,BI Developer,Fully Meets,IT/IS,4,0
6,BI Director,Fully Meets,IT/IS,1,0
7,CIO,Exceeds,IT/IS,1,0
8,Data Analyst,Exceeds,IT/IS,2,0
9,Data Analyst,Fully Meets,IT/IS,4,1


In [ ]:
#Position x EmploymentStatus x PerformanceScore x TerminatedFlag
df.pivot_table(
    index=['Position', 'EmploymentStatus', 'PerformanceScore'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0
).reset_index()

TerminatedFlag,Position,EmploymentStatus,PerformanceScore,0,1
0,Accountant I,Active,Fully Meets,3,0
1,Administrative Assistant,Active,Fully Meets,1,0
2,Administrative Assistant,Terminated for Cause,Fully Meets,0,1
3,Administrative Assistant,Voluntarily Terminated,Fully Meets,0,1
4,Area Sales Manager,Active,Exceeds,2,0
5,Area Sales Manager,Active,Fully Meets,18,0
6,Area Sales Manager,Active,PIP,3,0
7,Area Sales Manager,Terminated for Cause,Fully Meets,0,1
8,Area Sales Manager,Terminated for Cause,PIP,0,1
9,Area Sales Manager,Voluntarily Terminated,Fully Meets,0,2


In [ ]:
#Position × Sex × PerformanceScore × TerminatedFlag
df.pivot_table(
    index=['Position', 'Sex', 'PerformanceScore'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0
).reset_index()

TerminatedFlag,Position,Sex,PerformanceScore,0,1
0,Accountant I,F,Fully Meets,1,0
1,Accountant I,M,Fully Meets,2,0
2,Administrative Assistant,F,Fully Meets,1,2
3,Area Sales Manager,F,Exceeds,1,0
4,Area Sales Manager,F,Fully Meets,8,2
5,Area Sales Manager,F,PIP,1,0
6,Area Sales Manager,M,Exceeds,1,0
7,Area Sales Manager,M,Fully Meets,10,1
8,Area Sales Manager,M,PIP,2,1
9,BI Developer,F,Fully Meets,1,0


In [ ]:
#Position × Salary × Department × TerminatedFlag
df.groupby(
    ["Position", "Department", "TerminatedFlag"]
).agg(
    Count=("Salary", "count"),
    Avg_Salary9=("Salary", "mean"),
    Median_Salary9=("Salary", "median"),
    Min_Salary9=("Salary", "min"),
    Max_Salary9=("Salary", "max")
).reset_index()

,Position,Department,TerminatedFlag,Count,Avg_Salary9,Median_Salary9,Min_Salary9,Max_Salary9
0,Accountant I,Admin Offices,0,3,63507.666667,63003.0,63000,64520
1,Administrative Assistant,Admin Offices,0,1,51920.000000,51920.0,51920,51920
2,Administrative Assistant,Admin Offices,1,2,52460.000000,52460.0,49920,55000
3,Area Sales Manager,Sales,0,23,64243.000000,63291.0,55875,74241
4,Area Sales Manager,Sales,1,4,68897.500000,70947.0,59370,74326
5,BI Developer,IT/IS,0,4,95465.000000,96370.0,90100,99020
6,BI Director,IT/IS,0,1,110929.000000,110929.0,110929,110929
7,CIO,IT/IS,0,1,220450.000000,220450.0,220450,220450
8,Data Analyst,IT/IS,0,6,89405.833333,89587.5,83552,93554
9,Data Analyst,IT/IS,1,1,93093.000000,93093.0,93093,93093


In [ ]:
#Position × MaritalDesc × PerformanceScore × TerminatedFlag
df.pivot_table(
    index=['Position', 'MaritalDesc', 'PerformanceScore'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0
).reset_index()

TerminatedFlag,Position,MaritalDesc,PerformanceScore,0,1
0,Accountant I,Divorced,Fully Meets,1,0
1,Accountant I,Married,Fully Meets,1,0
2,Accountant I,Single,Fully Meets,1,0
3,Administrative Assistant,Married,Fully Meets,0,2
4,Administrative Assistant,Single,Fully Meets,1,0
5,Area Sales Manager,Married,Fully Meets,6,1
6,Area Sales Manager,Married,PIP,1,1
7,Area Sales Manager,Separated,Exceeds,1,0
8,Area Sales Manager,Separated,Fully Meets,1,0
9,Area Sales Manager,Single,Exceeds,1,0


In [ ]:
#Position × MaritalDesc × Department × TerminatedFlag
df.pivot_table(
    index=['Position', 'MaritalDesc', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0
).reset_index()

TerminatedFlag,Position,MaritalDesc,Department,0,1
0,Accountant I,Divorced,Admin Offices,1,0
1,Accountant I,Married,Admin Offices,1,0
2,Accountant I,Single,Admin Offices,1,0
3,Administrative Assistant,Married,Admin Offices,0,2
4,Administrative Assistant,Single,Admin Offices,1,0
5,Area Sales Manager,Married,Sales,7,2
6,Area Sales Manager,Separated,Sales,2,0
7,Area Sales Manager,Single,Sales,14,1
8,Area Sales Manager,Widowed,Sales,0,1
9,BI Developer,Married,IT/IS,3,0


In [ ]:
#Position x Sex x Department X TerminatedFlag
df.pivot_table(
    index=['Position', 'Sex', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0
).reset_index()

TerminatedFlag,Position,Sex,Department,0,1
0,Accountant I,F,Admin Offices,1,0
1,Accountant I,M,Admin Offices,2,0
2,Administrative Assistant,F,Admin Offices,1,2
3,Area Sales Manager,F,Sales,10,2
4,Area Sales Manager,M,Sales,13,2
5,BI Developer,F,IT/IS,1,0
6,BI Developer,M,IT/IS,3,0
7,BI Director,M,IT/IS,1,0
8,CIO,F,IT/IS,1,0
9,Data Analyst,F,IT/IS,3,0


In [ ]:
#Position x TermReason
df.groupby(
    ['Position', 'TermReason']).size().reset_index(name='Count')

,Position,TermReason,Count
0,Accountant I,N/A-StillEmployed,3
1,Administrative Assistant,N/A-StillEmployed,1
2,Administrative Assistant,career change,1
3,Administrative Assistant,"no-call, no-show",1
4,Area Sales Manager,Another position,1
5,Area Sales Manager,Fatal attraction,1
6,Area Sales Manager,N/A-StillEmployed,23
7,Area Sales Manager,attendance,1
8,Area Sales Manager,relocation out of area,1
9,BI Developer,N/A-StillEmployed,4


In [ ]:
#Create variable AgeGroup
df["AgeGroup"] = pd.cut(
    df["Age"],
    bins=[20, 30, 40, 50, 60, 100],
    labels=["20–29", "30–39", "40–49", "50–59", "60+"]
)

In [ ]:
#Age × PerformanceScore × Department × TerminatedFlag
df.pivot_table(
    index=['AgeGroup', 'PerformanceScore', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,AgeGroup,PerformanceScore,Department,0,1
0,20–29,Exceeds,Admin Offices,0,0
1,20–29,Exceeds,Executive Office,0,0
2,20–29,Exceeds,IT/IS,0,0
3,20–29,Exceeds,Production,0,0
4,20–29,Exceeds,Sales,0,0
5,20–29,Exceeds,Software Engineering,0,0
6,20–29,Fully Meets,Admin Offices,0,0
7,20–29,Fully Meets,Executive Office,0,0
8,20–29,Fully Meets,IT/IS,0,0
9,20–29,Fully Meets,Production,0,0


In [ ]:
#Age × EmploymentStatus × Department × TerminatedFlag
df.pivot_table(
    index=['AgeGroup', 'EmploymentStatus', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,AgeGroup,EmploymentStatus,Department,0,1
0,20–29,Active,Admin Offices,0,0
1,20–29,Active,Executive Office,0,0
2,20–29,Active,IT/IS,0,0
3,20–29,Active,Production,0,0
4,20–29,Active,Sales,0,0
5,20–29,Active,Software Engineering,0,0
6,20–29,Terminated for Cause,Admin Offices,0,0
7,20–29,Terminated for Cause,Executive Office,0,0
8,20–29,Terminated for Cause,IT/IS,0,0
9,20–29,Terminated for Cause,Production,0,0


In [ ]:
#Age × Sex × Department × TerminatedFlag
df.pivot_table(
    index=['AgeGroup', 'Sex', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,AgeGroup,Sex,Department,0,1
0,20–29,F,Admin Offices,0,0
1,20–29,F,Executive Office,0,0
2,20–29,F,IT/IS,0,0
3,20–29,F,Production,0,0
4,20–29,F,Sales,0,0
5,20–29,F,Software Engineering,0,0
6,20–29,M,Admin Offices,0,0
7,20–29,M,Executive Office,0,0
8,20–29,M,IT/IS,0,0
9,20–29,M,Production,0,0


In [ ]:
#Age × MaritalDesc × Department × TerminatedFlag
df.pivot_table(
    index=['AgeGroup', 'MaritalDesc', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,AgeGroup,MaritalDesc,Department,0,1
0,20–29,Divorced,Admin Offices,0,0
1,20–29,Divorced,Executive Office,0,0
2,20–29,Divorced,IT/IS,0,0
3,20–29,Divorced,Production,0,0
4,20–29,Divorced,Sales,0,0
5,20–29,Divorced,Software Engineering,0,0
6,20–29,Married,Admin Offices,0,0
7,20–29,Married,Executive Office,0,0
8,20–29,Married,IT/IS,0,0
9,20–29,Married,Production,0,0


In [ ]:
#AgeGroup × Salary × PerformanceScore × TerminatedFlag
df.groupby(
    ["AgeGroup", "PerformanceScore", "TerminatedFlag"], observed=False
).agg(
    Count=("Salary", "count"),
    Avg_Salary10=("Salary", "mean"),
    Median_Salary10=("Salary", "median"),
    Min_Salary10=("Salary", "min"),
    Max_Salary10=("Salary", "max")
).reset_index()

,AgeGroup,PerformanceScore,TerminatedFlag,Count,Avg_Salary10,Median_Salary10,Min_Salary10,Max_Salary10
0,20–29,Exceeds,0,0,NaN,NaN,NaN,NaN
1,20–29,Exceeds,1,0,NaN,NaN,NaN,NaN
2,20–29,Fully Meets,0,0,NaN,NaN,NaN,NaN
3,20–29,Fully Meets,1,0,NaN,NaN,NaN,NaN
4,20–29,Needs Improvement,0,0,NaN,NaN,NaN,NaN
5,20–29,Needs Improvement,1,0,NaN,NaN,NaN,NaN
6,20–29,PIP,0,0,NaN,NaN,NaN,NaN
7,20–29,PIP,1,0,NaN,NaN,NaN,NaN
8,30–39,Exceeds,0,7,71269.571429,63353.0,51777.0,93554.0
9,30–39,Exceeds,1,3,55416.666667,57954.0,49773.0,58523.0


In [ ]:
#Age × PerformanceScore × Position × TerminatedFlag
df.pivot_table(
    index=['AgeGroup', 'PerformanceScore', 'Position'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,AgeGroup,PerformanceScore,Position,0,1
0,20–29,Exceeds,Accountant I,0,0
1,20–29,Exceeds,Administrative Assistant,0,0
2,20–29,Exceeds,Area Sales Manager,0,0
3,20–29,Exceeds,BI Developer,0,0
4,20–29,Exceeds,BI Director,0,0
5,20–29,Exceeds,CIO,0,0
6,20–29,Exceeds,Data Analyst,0,0
7,20–29,Exceeds,Data Analyst,0,0
8,20–29,Exceeds,Data Architect,0,0
9,20–29,Exceeds,Database Administrator,0,0


In [ ]:
#AgeGroup × Salary × Department × TerminatedFlag
df.groupby(
    ["AgeGroup", "Department", "TerminatedFlag"], observed=False
).agg(
    Count=("Salary", "count"),
    Avg_Salary11=("Salary", "mean"),
    Median_Salary11=("Salary", "median"),
    Min_Salary11=("Salary", "min"),
    Max_Salary11=("Salary", "max")
).reset_index()

,AgeGroup,Department,TerminatedFlag,Count,Avg_Salary11,Median_Salary11,Min_Salary11,Max_Salary11
0,20–29,Admin Offices,0,0,NaN,NaN,NaN,NaN
1,20–29,Admin Offices,1,0,NaN,NaN,NaN,NaN
2,20–29,Executive Office,0,0,NaN,NaN,NaN,NaN
3,20–29,Executive Office,1,0,NaN,NaN,NaN,NaN
4,20–29,IT/IS,0,0,NaN,NaN,NaN,NaN
5,20–29,IT/IS,1,0,NaN,NaN,NaN,NaN
6,20–29,Production,0,0,NaN,NaN,NaN,NaN
7,20–29,Production,1,0,NaN,NaN,NaN,NaN
8,20–29,Sales,0,0,NaN,NaN,NaN,NaN
9,20–29,Sales,1,0,NaN,NaN,NaN,NaN


In [ ]:
#AgeGroup x TermReason
df.groupby(
    ['AgeGroup', 'TermReason'], observed=False).size().reset_index(name='Count')

,AgeGroup,TermReason,Count
0,20–29,Another position,0
1,20–29,Fatal attraction,0
2,20–29,Learned that he is a gangster,0
3,20–29,N/A-StillEmployed,0
4,20–29,attendance,0
5,20–29,career change,0
6,20–29,gross misconduct,0
7,20–29,hours,0
8,20–29,maternity leave - did not return,0
9,20–29,medical issues,0


In [ ]:
#Create variable TenureGroup
bins = [0, 2, 5, 10, 15, 20]
labels = ["0-2 yrs", "2-5 yrs", "5-10 yrs", "10-15 yrs", "15-20 yrs"]

df["TenureGroup"] = pd.cut(df["Tenure_Years"], bins=bins, labels=labels, include_lowest=True)

In [ ]:
#Tenure x PerformanceScore x Department x TerminatedFlag
df.pivot_table(
    index=['TenureGroup', 'PerformanceScore', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,TenureGroup,PerformanceScore,Department,0,1
0,0-2 yrs,Exceeds,Admin Offices,0,0
1,0-2 yrs,Exceeds,Executive Office,0,0
2,0-2 yrs,Exceeds,IT/IS,0,0
3,0-2 yrs,Exceeds,Production,0,1
4,0-2 yrs,Exceeds,Sales,0,0
5,0-2 yrs,Exceeds,Software Engineering,0,0
6,0-2 yrs,Fully Meets,Admin Offices,0,2
7,0-2 yrs,Fully Meets,Executive Office,0,0
8,0-2 yrs,Fully Meets,IT/IS,0,6
9,0-2 yrs,Fully Meets,Production,0,17


In [ ]:
#Tenure x EmploymentStatus x Department x TerminatedFlag
df.pivot_table(
    index=['TenureGroup', 'EmploymentStatus', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,TenureGroup,EmploymentStatus,Department,0,1
0,0-2 yrs,Active,Admin Offices,0,0
1,0-2 yrs,Active,Executive Office,0,0
2,0-2 yrs,Active,IT/IS,0,0
3,0-2 yrs,Active,Production,0,0
4,0-2 yrs,Active,Sales,0,0
5,0-2 yrs,Active,Software Engineering,0,0
6,0-2 yrs,Terminated for Cause,Admin Offices,0,1
7,0-2 yrs,Terminated for Cause,Executive Office,0,0
8,0-2 yrs,Terminated for Cause,IT/IS,0,2
9,0-2 yrs,Terminated for Cause,Production,0,0


In [ ]:
#Tenure x Sex x PerformanceScore x TerminatedFlag
df.pivot_table(
    index=['TenureGroup', 'Sex', 'PerformanceScore'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,TenureGroup,Sex,PerformanceScore,0,1
0,0-2 yrs,F,Exceeds,0,1
1,0-2 yrs,F,Fully Meets,0,17
2,0-2 yrs,F,Needs Improvement,0,0
3,0-2 yrs,F,PIP,0,0
4,0-2 yrs,M,Exceeds,0,0
5,0-2 yrs,M,Fully Meets,0,9
6,0-2 yrs,M,Needs Improvement,0,1
7,0-2 yrs,M,PIP,0,1
8,2-5 yrs,F,Exceeds,0,3
9,2-5 yrs,F,Fully Meets,0,20


In [ ]:
#Tenure × Salary × Department × TerminatedFlag
df.groupby(
    ["TenureGroup", "Department", "TerminatedFlag"], observed=False
).agg(
    Count=("Salary", "count"),
    Avg_Salary12=("Salary", "mean"),
    Median_Salary12=("Salary", "median"),
    Min_Salary12=("Salary", "min"),
    Max_Salary12=("Salary", "max")
).reset_index()

,TenureGroup,Department,TerminatedFlag,Count,Avg_Salary12,Median_Salary12,Min_Salary12,Max_Salary12
0,0-2 yrs,Admin Offices,0,0,NaN,NaN,NaN,NaN
1,0-2 yrs,Admin Offices,1,2,52460.000000,52460.0,49920.0,55000.0
2,0-2 yrs,Executive Office,0,0,NaN,NaN,NaN,NaN
3,0-2 yrs,Executive Office,1,0,NaN,NaN,NaN,NaN
4,0-2 yrs,IT/IS,0,0,NaN,NaN,NaN,NaN
5,0-2 yrs,IT/IS,1,6,97689.666667,98765.0,75281.0,114800.0
6,0-2 yrs,Production,0,0,NaN,NaN,NaN,NaN
7,0-2 yrs,Production,1,20,64212.850000,64395.0,46430.0,83082.0
8,0-2 yrs,Sales,0,0,NaN,NaN,NaN,NaN
9,0-2 yrs,Sales,1,1,59370.000000,59370.0,59370.0,59370.0


In [ ]:
#Tenure × Salary × Position × TerminatedFlag
df.groupby(
    ["TenureGroup", "Position", "TerminatedFlag"], observed=False
).agg(
    Count=("Salary", "count"),
    Avg_Salary13=("Salary", "mean"),
    Median_Salary13=("Salary", "median"),
    Min_Salary13=("Salary", "min"),
    Max_Salary13=("Salary", "max")
).reset_index()

,TenureGroup,Position,TerminatedFlag,Count,Avg_Salary13,Median_Salary13,Min_Salary13,Max_Salary13
0,0-2 yrs,Accountant I,0,0,NaN,NaN,NaN,NaN
1,0-2 yrs,Accountant I,1,0,NaN,NaN,NaN,NaN
2,0-2 yrs,Administrative Assistant,0,0,NaN,NaN,NaN,NaN
3,0-2 yrs,Administrative Assistant,1,2,52460.000000,52460.0,49920.0,55000.0
4,0-2 yrs,Area Sales Manager,0,0,NaN,NaN,NaN,NaN
5,0-2 yrs,Area Sales Manager,1,1,59370.000000,59370.0,59370.0,59370.0
6,0-2 yrs,BI Developer,0,0,NaN,NaN,NaN,NaN
7,0-2 yrs,BI Developer,1,0,NaN,NaN,NaN,NaN
8,0-2 yrs,BI Director,0,0,NaN,NaN,NaN,NaN
9,0-2 yrs,BI Director,1,0,NaN,NaN,NaN,NaN


In [ ]:
#Tenure × Salary × PerformanceScore × TerminatedFlag
df.groupby(
    ["TenureGroup", "PerformanceScore", "TerminatedFlag"], observed=False
).agg(
    Count=("Salary", "count"),
    Avg_Salary14=("Salary", "mean"),
    Median_Salary14=("Salary", "median"),
    Min_Salary14=("Salary", "min"),
    Max_Salary14=("Salary", "max")
).reset_index()

,TenureGroup,PerformanceScore,TerminatedFlag,Count,Avg_Salary14,Median_Salary14,Min_Salary14,Max_Salary14
0,0-2 yrs,Exceeds,0,0,NaN,NaN,NaN,NaN
1,0-2 yrs,Exceeds,1,1,57954.000000,57954.0,57954.0,57954.0
2,0-2 yrs,Fully Meets,0,0,NaN,NaN,NaN,NaN
3,0-2 yrs,Fully Meets,1,26,70182.730769,64839.5,46430.0,114800.0
4,0-2 yrs,Needs Improvement,0,0,NaN,NaN,NaN,NaN
5,0-2 yrs,Needs Improvement,1,1,83082.000000,83082.0,83082.0,83082.0
6,0-2 yrs,PIP,0,0,NaN,NaN,NaN,NaN
7,0-2 yrs,PIP,1,1,68898.000000,68898.0,68898.0,68898.0
8,2-5 yrs,Exceeds,0,0,NaN,NaN,NaN,NaN
9,2-5 yrs,Exceeds,1,6,66274.833333,60474.0,47434.0,108987.0


In [ ]:
#Tenure x Age x Department x TerminatedFlag
df.pivot_table(
    index=['TenureGroup', 'AgeGroup', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,TenureGroup,AgeGroup,Department,0,1
0,0-2 yrs,20–29,Admin Offices,0,0
1,0-2 yrs,20–29,Executive Office,0,0
2,0-2 yrs,20–29,IT/IS,0,0
3,0-2 yrs,20–29,Production,0,0
4,0-2 yrs,20–29,Sales,0,0
5,0-2 yrs,20–29,Software Engineering,0,0
6,0-2 yrs,30–39,Admin Offices,0,2
7,0-2 yrs,30–39,Executive Office,0,0
8,0-2 yrs,30–39,IT/IS,0,4
9,0-2 yrs,30–39,Production,0,7


In [ ]:
#Tenure x Age x Position x TerminatedFlag
df.pivot_table(
    index=['TenureGroup', 'AgeGroup', 'Position'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,TenureGroup,AgeGroup,Position,0,1
0,0-2 yrs,20–29,Accountant I,0,0
1,0-2 yrs,20–29,Administrative Assistant,0,0
2,0-2 yrs,20–29,Area Sales Manager,0,0
3,0-2 yrs,20–29,BI Developer,0,0
4,0-2 yrs,20–29,BI Director,0,0
5,0-2 yrs,20–29,CIO,0,0
6,0-2 yrs,20–29,Data Analyst,0,0
7,0-2 yrs,20–29,Data Analyst,0,0
8,0-2 yrs,20–29,Data Architect,0,0
9,0-2 yrs,20–29,Database Administrator,0,0


In [ ]:
#TenureGroup x TermReason
df.groupby(
    ['TenureGroup', 'TermReason'], observed=False).size().reset_index(name='Count')

,TenureGroup,TermReason,Count
0,0-2 yrs,Another position,6
1,0-2 yrs,Fatal attraction,0
2,0-2 yrs,Learned that he is a gangster,0
3,0-2 yrs,N/A-StillEmployed,0
4,0-2 yrs,attendance,1
5,0-2 yrs,career change,5
6,0-2 yrs,gross misconduct,0
7,0-2 yrs,hours,2
8,0-2 yrs,maternity leave - did not return,0
9,0-2 yrs,medical issues,1


In [ ]:
#ManagerName x Sex X EmploymentStatus X TerminatedFlag
df.pivot_table(
    index=['ManagerName', 'Sex', 'EmploymentStatus'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,ManagerName,Sex,EmploymentStatus,0,1
0,Alex Sweetwater,F,Active,5,0
1,Alex Sweetwater,F,Terminated for Cause,0,1
2,Alex Sweetwater,M,Active,1,0
3,Alex Sweetwater,M,Voluntarily Terminated,0,2
4,Amy Dunn,F,Active,5,0
5,Amy Dunn,F,Voluntarily Terminated,0,8
6,Amy Dunn,M,Active,3,0
7,Amy Dunn,M,Voluntarily Terminated,0,5
8,Board of Directors,F,Active,2,0
9,Brandon R. LeBlanc,F,Active,3,0


In [ ]:
#ManagerName x MaritalDesc X EmploymentStatus X TerminatedFlag
df.pivot_table(
    index=['ManagerName', 'MaritalDesc', 'EmploymentStatus'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,ManagerName,MaritalDesc,EmploymentStatus,0,1
0,Alex Sweetwater,Married,Active,1,0
1,Alex Sweetwater,Married,Terminated for Cause,0,1
2,Alex Sweetwater,Single,Active,5,0
3,Alex Sweetwater,Single,Voluntarily Terminated,0,2
4,Amy Dunn,Divorced,Voluntarily Terminated,0,1
5,Amy Dunn,Married,Active,4,0
6,Amy Dunn,Married,Voluntarily Terminated,0,6
7,Amy Dunn,Single,Active,4,0
8,Amy Dunn,Single,Voluntarily Terminated,0,5
9,Amy Dunn,Widowed,Voluntarily Terminated,0,1


In [ ]:
#ManagerName x Sex X MaritalDesc X TerminatedFlag
df.pivot_table(
    index=['ManagerName', 'Sex', 'MaritalDesc'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,ManagerName,Sex,MaritalDesc,0,1
0,Alex Sweetwater,F,Married,1,1
1,Alex Sweetwater,F,Single,4,0
2,Alex Sweetwater,M,Single,1,2
3,Amy Dunn,F,Divorced,0,1
4,Amy Dunn,F,Married,2,3
5,Amy Dunn,F,Single,3,4
6,Amy Dunn,M,Married,2,3
7,Amy Dunn,M,Single,1,1
8,Amy Dunn,M,Widowed,0,1
9,Board of Directors,F,Married,2,0


In [ ]:
#ManagerName x Department X EmploymentStatus X TerminatedFlag
df.pivot_table(
    index=['ManagerName', 'Department', 'EmploymentStatus'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,ManagerName,Department,EmploymentStatus,0,1
0,Alex Sweetwater,Software Engineering,Active,6,0
1,Alex Sweetwater,Software Engineering,Terminated for Cause,0,1
2,Alex Sweetwater,Software Engineering,Voluntarily Terminated,0,2
3,Amy Dunn,Production,Active,8,0
4,Amy Dunn,Production,Voluntarily Terminated,0,13
5,Board of Directors,Admin Offices,Active,1,0
6,Board of Directors,Executive Office,Active,1,0
7,Brandon R. LeBlanc,Admin Offices,Active,5,0
8,Brandon R. LeBlanc,Admin Offices,Terminated for Cause,0,1
9,Brandon R. LeBlanc,Admin Offices,Voluntarily Terminated,0,1


In [ ]:
#ManagerName x MaritalDesc X Department X TerminatedFlag
df.pivot_table(
    index=['ManagerName', 'MaritalDesc', 'Department'],
    columns='TerminatedFlag',
    aggfunc='size',
    fill_value=0,
    observed=False
).reset_index()

TerminatedFlag,ManagerName,MaritalDesc,Department,0,1
0,Alex Sweetwater,Married,Software Engineering,1,1
1,Alex Sweetwater,Single,Software Engineering,5,2
2,Amy Dunn,Divorced,Production,0,1
3,Amy Dunn,Married,Production,4,6
4,Amy Dunn,Single,Production,4,5
5,Amy Dunn,Widowed,Production,0,1
6,Board of Directors,Married,Admin Offices,1,0
7,Board of Directors,Married,Executive Office,1,0
8,Brandon R. LeBlanc,Divorced,Admin Offices,1,0
9,Brandon R. LeBlanc,Married,Admin Offices,2,2


In [ ]:
#ManagerName x TerminationRate x TermReason
top_manager_names = df["ManagerName"].tolist()

termination_reasons = (
    df[df["ManagerName"].isin(top_manager_names) & (df["TerminatedFlag"] == 1)]
    [["ManagerName", "TermReason"]]
    .groupby(["ManagerName", "TermReason"])
    .size()
    .reset_index(name="Count")
)

termination_reasons

,ManagerName,TermReason,Count
0,Alex Sweetwater,Another position,1
1,Alex Sweetwater,attendance,1
2,Alex Sweetwater,medical issues,1
3,Amy Dunn,Another position,1
4,Amy Dunn,attendance,1
5,Amy Dunn,hours,2
6,Amy Dunn,medical issues,1
7,Amy Dunn,more money,2
8,Amy Dunn,relocation out of area,1
9,Amy Dunn,return to school,1


In [ ]:
#Department x TermReason
df.groupby(['Department', 'TermReason']).size().reset_index(name='Count')

,Department,TermReason,Count
0,Admin Offices,N/A-StillEmployed,7
1,Admin Offices,career change,1
2,Admin Offices,"no-call, no-show",1
3,Executive Office,N/A-StillEmployed,1
4,IT/IS,Another position,1
5,IT/IS,Learned that he is a gangster,1
6,IT/IS,N/A-StillEmployed,40
7,IT/IS,career change,1
8,IT/IS,hours,2
9,IT/IS,medical issues,1


In [ ]:
#PerformanceScore x TermReason
df.groupby(['PerformanceScore', 'TermReason']).size().reset_index(name='Count')

,PerformanceScore,TermReason,Count
0,Exceeds,Another position,2
1,Exceeds,N/A-StillEmployed,29
2,Exceeds,gross misconduct,1
3,Exceeds,maternity leave - did not return,1
4,Exceeds,more money,2
5,Exceeds,retiring,1
6,Exceeds,unhappy,1
7,Fully Meets,Another position,17
8,Fully Meets,Learned that he is a gangster,1
9,Fully Meets,N/A-StillEmployed,162
